In [1]:
import tensorflow as tf # type: ignore
import numpy as np # type: ignore
import pandas as pd # type: ignore
import json
import string
import random
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore
from sklearn.preprocessing import LabelEncoder # type: ignore

# Load JSON data
with open('intents.json') as file:
    data = json.load(file)

patterns = []
tags = []
responses = {}
for intent in data['intents']:
    responses[intent['tag']] = intent['responses']
    for pattern in intent['patterns']:
        patterns.append(pattern)
        tags.append(intent['tag'])

data = pd.DataFrame({"pattern": patterns, "tags": tags})


In [2]:
# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=4000)
tokenizer.fit_on_texts(data['pattern'])
X_train = tokenizer.texts_to_sequences(data['pattern'])
X_train = pad_sequences(X_train, maxlen=20)

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])


In [3]:
# Define the input shape
sequence_length = 20  # Ensure this matches the maxlen used in padding
vocabulary_size = len(tokenizer.word_index) + 1  # +1 for the padding token
embedding_dim = 10
num_classes = len(le.classes_)

# Define the model
inputs = Input(shape=(sequence_length,))
x = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim)(inputs)
x = LSTM(10, return_sequences=True)(x)
x = Flatten()(x)
outputs = Dense(num_classes, activation="softmax")(x)

# Create and compile the model
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 embedding (Embedding)       (None, 20, 10)            2800      
                                                                 
 lstm (LSTM)                 (None, 20, 10)            840       
                                                                 
 flatten (Flatten)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 38)                7638      
                                                                 
Total params: 11,278
Trainable params: 11,278
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Generate dummy data for demonstration
X_train_dummy = np.random.randint(0, vocabulary_size, (1000, sequence_length))
y_train_dummy = np.random.randint(0, num_classes, 1000)

# Train the model
try:
    history = model.fit(X_train_dummy, y_train_dummy, epochs=10, batch_size=32)
except Exception as e:
    print(f"Error during training: {e}")


Epoch 1/10
32/32 [==============================] - 2s 9ms/step - loss: 3.6378 - accuracy: 0.0290
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 3.6295 - accuracy: 0.0430
Epoch 3/10
32/32 [==============================] - 0s 8ms/step - loss: 3.6180 - accuracy: 0.0410
Epoch 4/10
32/32 [==============================] - 0s 9ms/step - loss: 3.6027 - accuracy: 0.0390
Epoch 5/10
32/32 [==============================] - 0s 9ms/step - loss: 3.5762 - accuracy: 0.0430
Epoch 6/10
32/32 [==============================] - 0s 7ms/step - loss: 3.5340 - accuracy: 0.0640
Epoch 7/10
32/32 [==============================] - 0s 8ms/step - loss: 3.4720 - accuracy: 0.0880
Epoch 8/10
32/32 [==============================] - 0s 9ms/step - loss: 3.4235 - accuracy: 0.0900
Epoch 9/10
32/32 [==============================] - 0s 8ms/step - loss: 3.3731 - accuracy: 0.1090
Epoch 10/10
32/32 [==============================] - 0s 10ms/step - loss: 3.3115 - accuracy: 0.1210


In [6]:
while True:
    prediction_input = input("You: ")
    
    # Preprocess the input
    prediction_input = prediction_input.lower()
    prediction_input = ''.join(char for char in prediction_input if char not in string.punctuation)
    
    # Tokenize and pad the input
    prediction_input_seq = tokenizer.texts_to_sequences([prediction_input])
    prediction_input_padded = pad_sequences(prediction_input_seq, maxlen=sequence_length)
    
    # Predict and get the output
    try:
        output = model.predict(prediction_input_padded)
        predicted_tag = le.inverse_transform([output.argmax()])[0]
        
        # Output a response
        print("Bot:", random.choice(responses.get(predicted_tag, ["Sorry, I don't understand."])))
        
        if predicted_tag == 'goodbye':
            break
    except Exception as e:
        print(f"Error during prediction: {e}")


1/1 [==============================] - 0s 14ms/step
Bot: ENTER YOUR OWN UNIVERSITY UNIFORM CIRCULER
1/1 [==============================] - 0s 28ms/step
Bot: ENTER YOUR OWN UNIVERSITY UNIFORM CIRCULER
1/1 [==============================] - 0s 11ms/step
Bot: ENTER YOUR OWN UNIVERSITY UNIFORM CIRCULER
